In [7]:
import sys
sys.path.insert(0, "../cambiame-app/")

from pathlib import Path
import geemap

from src.config import STATIC_FILES_PATH
from src.main import CambiumTakeHomeChallenge

In [9]:
# Instantiate the main class to download and render all data
cambium_challenge = CambiumTakeHomeChallenge(
    area_of_interest_geojson_filename="corrientes1_area", area_of_interest_geojson_file="../data/corrientes1/corrientes1_area.geojson"
)
# Plot different layers in geemap
m = geemap.Map()
m.add_basemap("SATELLITE")
m.add_layer(cambium_challenge.drainage_ee,
            vis_params={
                "palette": cambium_challenge.drainage_palette,
                "vmin": cambium_challenge.drainage_params.get("min"),
                "vmax": cambium_challenge.drainage_params.get("max")
            },
            name="Drainage",
            shown=False)
m.add_layer(cambium_challenge.land_usage_ee.first(), name="Land Usage", shown=False)
# m.add_geojson(cambium_challenge.argentina_protected_area.__geo_interface__, layer_name='Protected Areas')
elevation_dataset = cambium_challenge.elevation_xarray.assign_coords(time=('time', [0]))
geemap.xee_to_image(elevation_dataset, filenames=[Path(STATIC_FILES_PATH, "elevation.tif")])
for layer in [
    {"var": "elevation", "name": "Elevation"},
    # {"var": "slope", "name": "Slope"},
    # {"var": "binary_slope", "name": "Binary Slope (|slope| > 1)"}
]:
    var_raster = geemap.array_to_image(elevation_dataset[layer.get("var")],
                                       source=Path(STATIC_FILES_PATH, "elevation.tif"))
    m.add_raster(var_raster, colormap="terrain", layer_name=layer.get("name"), shown=False)
# m.add_geojson(cambium_challenge.area_of_interest_geojson_file.__geo_interface__,
#               layer_name="Area of Interest",
#               color="red")
# Get areas with the study applied ;)
areas_of_study = cambium_challenge.get_area_viability()
m.add_geojson(areas_of_study.__geo_interface__,
              layer_name="Studied Areas",
              fill_colors=areas_of_study["color"])

Processing 1/1: 0


In [10]:
m

Map(center=[-28.5, -56.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…